In [56]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Read Data from Snowflake using Fosforml SDK

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
#table_name = 'YOUR_SNOWFLAKE_TABLE_NAME'

#sf_df = my_session.sql("select * from {}".format(table_name))
#df = sf_df.to_pandas()

In [4]:
df = pd.read_csv('Attendance_dataset.csv')

# Data Exploration

In [6]:
df.shape

(1000, 5)

In [10]:
df['Attendance_Date'] = pd.to_datetime(df['Attendance_Date'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Employee_Id      1000 non-null   object        
 1   Employee_name    1000 non-null   object        
 2   Attendance_Date  1000 non-null   datetime64[ns]
 3   Present_Status   1000 non-null   object        
 4   Hours_Worked     1000 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 39.2+ KB


In [13]:
df.describe()

,Attendance_Date,Hours_Worked
count,1000,1000.000000
mean,2023-02-14 13:00:28.799999744,2.253000
min,2023-01-01 00:00:00,0.000000
25%,2023-01-24 00:00:00,0.000000
50%,2023-02-15 00:00:00,0.000000
75%,2023-03-07 00:00:00,5.000000
max,2023-03-31 00:00:00,8.000000
std,NaN,2.846593


In [14]:
df.isnull().sum()

Employee_Id        0
Employee_name      0
Attendance_Date    0
Present_Status     0
Hours_Worked       0
dtype: int64

# Feature Engineering

In [15]:
df['Present_Status'] = df['Present_Status'].map({'Yes': 1, 'No': 0})

In [16]:
total_days_present = df.groupby('Employee_Id')['Present_Status'].sum().reset_index()
total_days_present.columns = ['Employee_Id', 'Total_Days_Present']

In [17]:
total_hours_worked = df.groupby('Employee_Id')['Hours_Worked'].sum().reset_index()
total_hours_worked.columns = ['Employee_Id', 'Total_Hours_Worked']

In [18]:
average_hours_per_day = df.groupby('Employee_Id')['Hours_Worked'].mean().reset_index()
average_hours_per_day.columns = ['Employee_Id', 'Average_Hours_Per_Day']

In [19]:
total_days = df['Attendance_Date'].nunique()
attendance_rate = total_days_present.copy()
attendance_rate['Attendance_Rate'] = (attendance_rate['Total_Days_Present'] / total_days) * 100

In [20]:
threshold_hours = 8
days_worked_more_than_x_hours = df[df['Hours_Worked'] > threshold_hours].groupby('Employee_Id').size().reset_index(name='Days_Worked_More_Than_X_Hours')

In [21]:
features = total_days_present.merge(total_hours_worked, on='Employee_Id')
features = features.merge(average_hours_per_day, on='Employee_Id')
features = features.merge(attendance_rate[['Employee_Id', 'Attendance_Rate']], on='Employee_Id')
features = features.merge(days_worked_more_than_x_hours, on='Employee_Id', how='left').fillna(0)

In [22]:
final_df = df[['Employee_Id', 'Employee_name']].drop_duplicates().merge(features, on='Employee_Id')

In [23]:
df['Day_of_Week'] = df['Attendance_Date'].dt.day_name()

In [24]:
df['Is_Weekend'] = df['Day_of_Week'].isin(['Saturday', 'Sunday']).astype(int)

In [25]:
df['Week_of_Month'] = df['Attendance_Date'].apply(lambda x: (x.day - 1) // 7 + 1)

In [26]:
df['Month'] = df['Attendance_Date'].dt.month

In [27]:
df['Quarter'] = df['Attendance_Date'].dt.quarter

In [28]:
start_date = df['Attendance_Date'].min()
df['Days_Since_Start'] = (df['Attendance_Date'] - start_date).dt.days

In [29]:
df['Cumulative_Days_Present'] = df.sort_values('Attendance_Date').groupby('Employee_Id')['Present_Status'].cumsum()

In [30]:
df['Rolling_Avg_Hours_Worked'] = df.groupby('Employee_Id')['Hours_Worked'].rolling(window=7, min_periods=1).mean().reset_index(level=0, drop=True)

In [31]:
final_df = df.copy()

In [32]:
final_df.shape

(1000, 13)

In [33]:
final_df.head()

,Employee_Id,Employee_name,Attendance_Date,Present_Status,Hours_Worked,Day_of_Week,Is_Weekend,Week_of_Month,Month,Quarter,Days_Since_Start,Cumulative_Days_Present,Rolling_Avg_Hours_Worked
0,E0044,Matthew Nelson,2023-01-01,0,0,Sunday,1,1,1,1,0,0,0.0
1,E0071,Kenneth Clark,2023-01-01,0,0,Sunday,1,1,1,1,0,0,0.0
2,E0059,Jennifer Kane,2023-01-01,0,0,Sunday,1,1,1,1,0,0,0.0
3,E0036,Jill Bush,2023-01-01,0,0,Sunday,1,1,1,1,0,0,0.0
4,E0024,Madison Edwards,2023-01-02,0,0,Monday,0,1,1,1,1,0,0.0


# Model Training

In [46]:
# Assuming df is your dataframe with all the engineered features
df = final_df.copy()

In [47]:
# Handle missing values if any
df.fillna(0, inplace=True)

In [48]:
# Define the target variable and features
X = df.drop(columns=['Hours_Worked', 'Employee_name', 'Attendance_Date','Employee_Id','Present_Status','Day_of_Week'])
y = df['Hours_Worked']

In [49]:
X.columns = X.columns.str.upper()

In [50]:
y.name = 'HOURS_WORKED'

In [51]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [53]:
# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

In [57]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

In [58]:
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R2 Score: {r2}')

Mean Absolute Error: 0.8299850000000001
Root Mean Squared Error: 1.4625459166736219
R2 Score: 0.7219416194573889


# Tunning the Model